In [1]:
#from IPython.display import Image
%matplotlib inline

In [2]:
import ee
import datetime
ee.Initialize()

In [3]:
startTime = datetime.datetime(2013, 1, 1)
endTime = datetime.datetime(2015, 1, 1)
point = {'type':'Point', 'coordinates':[ -122.144622,37.414345]}; 
L8_DN = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate(startTime, endTime)
L8_DN = L8_DN.filterBounds(point)

In [30]:
def TIRtoLST(tir, emissivity) : 
    # computes emissivity-corrected land surface temperature given Landsat8 band 10 (tir) and emissivity
    # assumes tir, emissivity are google earth engine images
    k1 = 774.8853 ; # constants for Landsat8 B10
    k2 =  1321.0789 ; # constants for Landsat8 B10
    dummyk1 = emissivity.multiply(0).add(1).multiply(k1) # make dummy variable same size/type as emissivity
    dummyk2 = emissivity.multiply(0).add(1).multiply(k2) # make dummy variable same size/type as emissivity
    
    # first convert band 10 Digital Number (DN) data to TOA spectral radiance
    radiance = tir.multiply(3.3420E-04).add(.1)  # scale by gain and bias, band specific 
    L_lamda = radiance.divide(emissivity); # correct for emissivity
    k1_over_L_lamda = dummyk1.divide(L_lamda) 
    denominator = k1_over_L_lamda.add(1).log(); 
    temp = dummyk2.divide(denominator).subtract(273.15); # finally convert to degrees C
    return temp


In [31]:
startTime = datetime.datetime(2013, 1, 1)
endTime = datetime.datetime(2015, 1, 1)
point = {'type':'Point', 'coordinates':[ -122.144622,37.414345]}; 

L8_DN = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate(startTime, endTime)
L8_DN = L8_DN.filterBounds(point)
L8_DN = L8_DN.sort('CLOUD_COVER').first()

image = ee.Image(L8_DN)
tir = image.select(u'B10')
red = image.select(u'B4')
nir = image.select(u'B5')
ndvi = nir.subtract(red).divide(nir.add(red))

emissivity = ee.Image('NASA/ASTER_GED/AG100_003').select('emissivity_band10')

# #Compute Brigthness Temp
# radiance = tir.multiply(3.3420E-04).add(0.10000); 
# k1 = radiance.multiply(0).add(1).multiply(774.8853);
# l8_calc = k1.divide(radiance).add(1).log()
# k2 = l8_calc.multiply(0).add(1).multiply(1321.0789)
# T_B = k2.divide(l8_calc).subtract(273.15);

# #Computer Brightness Temp to Land Surface Temp
# a = T_B.multiply(10.8)
# b = a.divide(0.01438)
# c = b.add(1)
# denominator = c.multiply(emissivity.log())
# t = T_B.divide(denominator)
t = TIRtoLST(tir, emissivity)

In [32]:
info = t.getInfo()

In [36]:
t.getThumbURL()

u'https://earthengine.googleapis.com/api/thumb?thumbid=5763bfa01a284d56ac2d4359e7808353&token=4bb3a703ef198cd8ff8ba6f76cf9ee75'

In [27]:
t.reduceRegion('reducer')

In [9]:
point = {'type':'Point', 'coordinates':[ -122.144622,37.414345]}; 
info = t.getInfo() #t.getRegion(point,500).getInfo()

In [51]:
point = ee.Geometry.Point(-122.0918, 37.422)
region_bay_area = point.buffer(50000).bounds().getInfo()['coordinates']

In [52]:
Image(url=t.getThumbUrl({'region':region_bay_area}))

EEException: User memory limit exceeded.

In [36]:
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2013-01-01', '2016-01-01')
l8 = l8.select(u'B10')
l8 = l8.median()

radiance = l8.multiply(3.3420E-04).add(0.10000); 
k1 = radiance.multiply(0).add(1).multiply(774.8853);
l8_calc = k1.divide(radiance).add(1).log()
k2 = l8_calc.multiply(0).add(1).multiply(1321.0789)
temp = k2.divide(l8_calc).subtract(273.15);

In [31]:
point = ee.Geometry.Point(-122.0918, 37.422)
region_bay_area = point.buffer(50000).bounds().getInfo()['coordinates']

In [38]:
Image(url=temp.getThumbUrl({'region':region_bay_area}))

In [4]:
import datetime
from matplotlib import dates
import matplotlib.dates as mdates
#from pylab import *

In [19]:
# Create a reference to the image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2013-05-01', '2015-05-15')
l8 = l8.select(u'B5')
l8 = l8.median()


point = ee.Geometry.Point(-122.0918, 37.422)
region_bay_area = point.buffer(50000).bounds().getInfo()['coordinates']

info = l8.getInfo()



# And finally display the image.
Image(url=l8.getThumbUrl({'region':region_bay_area}))

In [26]:
whos

Variable            Type        Data/Info
-----------------------------------------
Image               type        <class 'IPython.core.display.Image'>
dates               module      <module 'matplotlib.dates<...>es/matplotlib/dates.pyc'>
datetime            module      <module 'datetime' from '<...>lib-dynload/datetime.so'>
ee                  module      <module 'ee' from '/opt/c<...>ackages/ee/__init__.pyc'>
info                dict        n=2
l8                  Image       ee.Image({\n  "type": "In<...>ame": "reduce.median"\n})
mdates              module      <module 'matplotlib.dates<...>es/matplotlib/dates.pyc'>
point               Geometry    ee.Geometry({\n  "type": <...>18, \n    37.422\n  ]\n})
region_bay_area     list        n=1
region_california   list        n=1
